In [24]:
import pandas as pd
import requests
import numpy as np
import os

pd.set_option('display.max_columns', None)


In [28]:
import pandas as pd
import requests
import time
import json

def fetch_worldbank_indicator(indicator, max_pages=20):
    """
    Faster and safer WB fetcher.
    - Stops after `max_pages`
    - Skips empty pages
    - Handles HTML and errors
    """
    all_rows = []

    for page in range(1, max_pages + 1):
        url = f"https://api.worldbank.org/v2/country/ALL/indicator/{indicator}?format=json&per_page=2000&page={page}"
        
        try:
            response = requests.get(url, timeout=10)
            response_json = response.json()
        except Exception as e:
            print(f"⚠️ Error on page {page} for {indicator}: {e}")
            break

        # Bad or empty response
        if not isinstance(response_json, list) or len(response_json) < 2:
            break
        
        data = response_json[1]
        
        # No more data on next pages
        if not data:
            break

        all_rows.extend(data)

        # Check if last page reached
        if response_json[0]["page"] >= response_json[0]["pages"]:
            break
        
        time.sleep(0.1)

    if len(all_rows) == 0:
        print(f"⚠️ No usable data for indicator: {indicator}")
        return pd.DataFrame(columns=["country", "countryiso3code", "year", indicator])

    df = pd.DataFrame(all_rows)[["country", "countryiso3code", "date", "value"]]
    df.rename(columns={"date": "year", "value": indicator}, inplace=True)

    return df


In [26]:
indicators = {
    "EN.ATM.PM25.MC.M3": "pm25",
    "EN.ATM.PM25.MC.ZS": "pm25_pop_weighted",

    # Updated GHG indicators
    "EN.GHG.CO2.MT.CE.AR5": "co2_total_mt",
    "EN.GHG.CH4.MT.CE.AR5": "ch4_total_mt",

    "AG.LND.FRST.ZS": "forest_area",
    "SP.URB.TOTL.IN.ZS": "urban_pop",
    "NY.GDP.PCAP.CD": "gdp_pc",
    "SP.DYN.LE00.IN": "life_expectancy",
    "SH.STA.AIRP.P5": "resp_mortality",
    "SP.POP.TOTL": "population"
}

dfs = []
print("Indicators loaded. Total =", len(indicators))


Indicators loaded. Total = 10


In [29]:
for code, colname in indicators.items():
    print(f"Fetching {colname} ({code}) ...")
    df = fetch_worldbank_indicator(code)
    df.rename(columns={code: colname}, inplace=True)
    dfs.append(df)

print("\nFetching completed for all indicators.")


Fetching pm25 (EN.ATM.PM25.MC.M3) ...
Fetching pm25_pop_weighted (EN.ATM.PM25.MC.ZS) ...
⚠️ Error on page 2 for EN.ATM.PM25.MC.ZS: HTTPSConnectionPool(host='api.worldbank.org', port=443): Read timed out. (read timeout=10)
Fetching co2_total_mt (EN.GHG.CO2.MT.CE.AR5) ...
Fetching ch4_total_mt (EN.GHG.CH4.MT.CE.AR5) ...
Fetching forest_area (AG.LND.FRST.ZS) ...
⚠️ Error on page 8 for AG.LND.FRST.ZS: HTTPSConnectionPool(host='api.worldbank.org', port=443): Read timed out. (read timeout=10)
Fetching urban_pop (SP.URB.TOTL.IN.ZS) ...
⚠️ Error on page 1 for SP.URB.TOTL.IN.ZS: HTTPSConnectionPool(host='api.worldbank.org', port=443): Read timed out. (read timeout=10)
⚠️ No usable data for indicator: SP.URB.TOTL.IN.ZS
Fetching gdp_pc (NY.GDP.PCAP.CD) ...
⚠️ Error on page 1 for NY.GDP.PCAP.CD: HTTPSConnectionPool(host='api.worldbank.org', port=443): Read timed out. (read timeout=10)
⚠️ No usable data for indicator: NY.GDP.PCAP.CD
Fetching life_expectancy (SP.DYN.LE00.IN) ...
⚠️ Error on page 1 f

In [34]:
import pandas as pd
import requests, zipfile, io

def load_wb_bulk(indicator):
    """
    Load World Bank indicator using bulk ZIP (CSV).
    Works reliably and identifies the correct data file automatically.
    """
    print(f"Downloading bulk data for {indicator} ...")
    url = f"https://api.worldbank.org/v2/en/indicator/{indicator}?downloadformat=csv"
    
    response = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(response.content))

    # Find the correct CSV file (it always starts with "API_" and ends with ".csv")
    data_file = None
    for name in z.namelist():
        if name.startswith("API_") and name.endswith(".csv"):
            data_file = name
            break

    if data_file is None:
        raise Exception("Could not find API_*.csv in ZIP file")

    df = pd.read_csv(z.open(data_file), skiprows=4)
    return df


In [35]:

dfs = {}

for code, name in indicators.items():
    print(f"\nDownloading {name} ({code}) ...")
    try:
        df = load_wb_bulk(code)
        dfs[name] = df
        print(f"✔ Loaded: {name}")
    except Exception as e:
        print(f"❌ Failed to load {name}: {e}")



✔ Loaded: pm25

✔ Loaded: pm25_pop_weighted

✔ Loaded: co2_total_mt

✔ Loaded: ch4_total_mt

✔ Loaded: forest_area

✔ Loaded: urban_pop

✔ Loaded: gdp_pc

✔ Loaded: life_expectancy

✔ Loaded: resp_mortality

✔ Loaded: population


In [38]:
cleaned = {}

for name, df in dfs.items():
    id_vars = ["Country Name", "Country Code"]
    year_cols = [col for col in df.columns if col.isdigit()]

    df_melt = df.melt(
        id_vars=id_vars,
        value_vars=year_cols,
        var_name="year",
        value_name=name
    )

    df_melt["year"] = df_melt["year"].astype(int)
    cleaned[name] = df_melt

print("✔ All datasets normalized")


✔ All datasets normalized


In [39]:
from functools import reduce

dfs_list = list(cleaned.values())

final_df = reduce(
    lambda left, right: pd.merge(
        left, right, on=["Country Name", "Country Code", "year"], how="outer"
    ),
    dfs_list
)

print("Merged shape:", final_df.shape)
final_df.head()


Merged shape: (17290, 13)


,Country Name,Country Code,year,pm25,pm25_pop_weighted,co2_total_mt,ch4_total_mt,forest_area,urban_pop,gdp_pc,life_expectancy,resp_mortality,population
0,Afghanistan,AFG,1960,NaN,NaN,NaN,NaN,NaN,8.401,NaN,32.799,NaN,9035043.0
1,Afghanistan,AFG,1961,NaN,NaN,NaN,NaN,NaN,8.684,NaN,33.291,NaN,9214083.0
2,Afghanistan,AFG,1962,NaN,NaN,NaN,NaN,NaN,8.976,NaN,33.757,NaN,9404406.0
3,Afghanistan,AFG,1963,NaN,NaN,NaN,NaN,NaN,9.276,NaN,34.201,NaN,9604487.0
4,Afghanistan,AFG,1964,NaN,NaN,NaN,NaN,NaN,9.586,NaN,34.673,NaN,9814318.0


In [40]:
final_df.rename(columns={"Country Name": "country", "Country Code": "iso3"}, inplace=True)

num_cols = [c for c in final_df.columns if c not in ["country", "iso3", "year"]]
final_df[num_cols] = final_df[num_cols].apply(pd.to_numeric, errors='coerce')


In [41]:
final_df["co2_pc"] = (final_df["co2_total_mt"] * 1_000_000) / final_df["population"]
final_df["ch4_pc"] = (final_df["ch4_total_mt"] * 1_000_000) / final_df["population"]


In [42]:
import os
os.makedirs("../data/cleaned", exist_ok=True)

final_df.to_csv("../data/cleaned/worldbank_bulk_cleaned.csv", index=False)
print("Saved final dataset to data/cleaned/worldbank_bulk_cleaned.csv")


Saved final dataset to data/cleaned/worldbank_bulk_cleaned.csv
